# Итоговая работа по курсу BIG DATA (Python)

### Задание. 
В данном разделе мы выступим в роли data scientist и попытаемся построить простую модель для рекомендации фильмов пользователям.  

***1. Загрузите в колаб файлы по оценкам (ratings) и фильмам (movies) и создайте на их основе pandas-датафреймы.  *** 

Сформировав общий топ фильмов в прошлой практике, мы хотим сделать шаг вперед и начать советовать пользователю те фильмы, которые могли бы быть для него наиболее интересны. Наша цель - научится предсказывать оценку фильма пользователем. Для тестирования модели найдем пользователя, который поставил больше всего оценок.  

***2. Средствами Pandas, используя dataframe ratings, найдите id пользователя, поставившего больше всего оценок. ***

Отберем фильмы, которые оценил данный пользователь.  

***3. Оставьте в датафрейме ratings только те фильмы, который оценил данный пользователь. *** 

Для построения модели нам нужны признаки. В качестве таковых будем использовать:  

●	Год выхода  
●	Жанры  
●	Общее количество оценок  
●	Суммарную оценку  

***4. Добавьте к датафрейму из задания 3 столбцы: ***  
 
●	По жанрам. Каждый столбец - это жанр. Единицу записываем, если фильм принадлежит данному жанру и 0 - если нет.  
●	Столбцы с общим количеством оценок от всех пользователей на фильм и суммарной оценкой от всех пользователей.   

Теперь все готово и можно строить модель.  

***5. Сформируйте X_train, X_test, y_train, y_test.  ***

***6. Возьмите модель линейной регрессии (или любую другую для задачи регрессии) и обучите ее на фильмах.  ***

***7. Оцените качество модели на X_test, y_test при помощи метрик для задачи регрессии.  ***


## 1. Загрузка данных

In [1]:
import pandas as pd

In [10]:
ratings = pd.read_csv('u.data.csv', names = ['User_id', 'Film_id', 'Rating', 'Timestamp'], sep='\t')
ratings

,User_id,Film_id,Rating,Timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [11]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   User_id    100000 non-null  int64
 1   Film_id    100000 non-null  int64
 2   Rating     100000 non-null  int64
 3   Timestamp  100000 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB


In [12]:
ratings.describe()

,User_id,Film_id,Rating,Timestamp
count,100000.00000,100000.000000,100000.000000,1.000000e+05
mean,462.48475,425.530130,3.529860,8.835289e+08
std,266.61442,330.798356,1.125674,5.343856e+06
min,1.00000,1.000000,1.000000,8.747247e+08
25%,254.00000,175.000000,3.000000,8.794487e+08
50%,447.00000,322.000000,4.000000,8.828269e+08
75%,682.00000,631.000000,4.000000,8.882600e+08
max,943.00000,1682.000000,5.000000,8.932866e+08


In [13]:
movies = pd.read_csv('u.item.csv', names = ['Film_id', 'Film_name', 'Release_date', 'Video_date', 'IMDb URL', '?', 'Action', 'Adventure', 'Animation', 'Child', 'Comedy', 'Detective', 'Docs',\
                                            'Drama', 'Fantasy', 'Noir', 'Horror', 'Musical', 'Mystiс', 'Romantic', 'Science', 'Thriller', 'War', 'Western'], sep='|', engine = 'python')
movies

,Film_id,Film_name,Release_date,Video_date,IMDb URL,?,Action,Adventure,Animation,Child,Comedy,Detective,Docs,Drama,Fantasy,Noir,Horror,Musical,Mystiс,Romantic,Science,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
from datetime import datetime
movies['Release_year'] = pd.DatetimeIndex(movies['Release_date']).year
movies['Release_year'] = movies.Release_year.astype('Int64')
movies = movies[['Film_id', 'Film_name', 'Release_year', 'Action', 'Adventure', 'Animation', 'Child', 'Comedy', 'Detective', 'Docs',\
                 'Drama', 'Fantasy', 'Noir', 'Horror', 'Musical', 'Mystiс', 'Romantic', 'Science', 'Thriller', 'War', 'Western']]
movies

,Film_id,Film_name,Release_year,Action,Adventure,Animation,Child,Comedy,Detective,Docs,Drama,Fantasy,Noir,Horror,Musical,Mystiс,Romantic,Science,Thriller,War,Western
0,1,Toy Story (1995),1995,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),1995,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),1995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),1995,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),1995,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),1998,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),1998,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),1998,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),1994,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Film_id       1682 non-null   int64 
 1   Film_name     1682 non-null   object
 2   Release_year  1681 non-null   Int64 
 3   Action        1682 non-null   int64 
 4   Adventure     1682 non-null   int64 
 5   Animation     1682 non-null   int64 
 6   Child         1682 non-null   int64 
 7   Comedy        1682 non-null   int64 
 8   Detective     1682 non-null   int64 
 9   Docs          1682 non-null   int64 
 10  Drama         1682 non-null   int64 
 11  Fantasy       1682 non-null   int64 
 12  Noir          1682 non-null   int64 
 13  Horror        1682 non-null   int64 
 14  Musical       1682 non-null   int64 
 15  Mystiс        1682 non-null   int64 
 16  Romantic      1682 non-null   int64 
 17  Science       1682 non-null   int64 
 18  Thriller      1682 non-null   int64 
 19  War   

In [17]:
movies.describe()

,Film_id,Release_year,Action,Adventure,Animation,Child,Comedy,Detective,Docs,Drama,Fantasy,Noir,Horror,Musical,Mystiс,Romantic,Science,Thriller,War,Western
count,1682.000000,1681.00000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.00000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000
mean,841.500000,1989.38608,0.149227,0.080262,0.024970,0.072533,0.300238,0.064804,0.029727,0.431034,0.01308,0.014269,0.054697,0.033294,0.036266,0.146849,0.060048,0.149227,0.042212,0.016052
std,485.695893,14.25358,0.356418,0.271779,0.156081,0.259445,0.458498,0.246253,0.169882,0.495368,0.11365,0.118632,0.227455,0.179456,0.187008,0.354061,0.237646,0.356418,0.201131,0.125714
min,1.000000,1922.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,421.250000,1993.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,841.500000,1995.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1261.750000,1996.00000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1682.000000,1998.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## 2. Пользователь, поставивший больше всего оценок

In [18]:
most_active_user = ratings.groupby('User_id').count()[['Timestamp']].sort_values('Timestamp', ascending=False).reset_index()
most_active_user.head(1)

,User_id,Timestamp
0,405,737


### или так

In [19]:
most_active_user_df = ratings.User_id.value_counts().reset_index().head(1)
most_active_user_df.columns = ['User_id', 'Rating']
most_active_user_df

,User_id,Rating
0,405,737


## 3. Фильмы, которые оценил пользователь 405

In [20]:
films_user405 = ratings.loc[ratings.User_id == 405]
films_names_user405 = films_user405.merge(movies, how='left', on='Film_id')
films_names_user405 = films_names_user405[['Film_id','Film_name', 'Release_year', 'Rating']]
films_names_user405['Release_year'] = films_names_user405.Release_year.astype('int64')
films_names_user405

,Film_id,Film_name,Release_year,Rating
0,56,Pulp Fiction (1994),1994,4
1,592,True Crime (1995),1995,1
2,1582,T-Men (1947),1947,1
3,171,Delicatessen (1991),1991,1
4,580,"Englishman Who Went Up a Hill, But Came Down a...",1995,1
...,...,...,...,...
732,375,Showgirls (1995),1995,1
733,445,"Body Snatcher, The (1945)",1945,4
734,1246,Bushwhacked (1995),1995,1
735,196,Dead Poets Society (1989),1989,1


In [21]:
films_names_user405.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 737 entries, 0 to 736
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Film_id       737 non-null    int64 
 1   Film_name     737 non-null    object
 2   Release_year  737 non-null    int64 
 3   Rating        737 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 28.8+ KB


## 4. Формирование df для построения модели

In [23]:
ratings_pivot_table = ratings.groupby('Film_id').agg({'Rating': ['sum', 'count']})['Rating']\
                             .sort_values('Film_id', ascending=True).reset_index()
ratings_pivot_table

,Film_id,sum,count
0,1,1753,452
1,2,420,131
2,3,273,90
3,4,742,209
4,5,284,86
...,...,...,...
1677,1678,1,1
1678,1679,3,1
1679,1680,2,1
1680,1681,3,1


In [24]:
ratings_pivot_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   Film_id  1682 non-null   int64
 1   sum      1682 non-null   int64
 2   count    1682 non-null   int64
dtypes: int64(3)
memory usage: 39.5 KB


In [25]:
genre_df = movies[['Film_id', 'Action', 'Adventure', 'Animation', 'Child', 'Comedy', 'Detective', 'Docs', 'Drama', 'Fantasy',\
                   'Noir', 'Horror', 'Musical', 'Mystiс', 'Romantic', 'Science', 'Thriller', 'War', 'Western']]
model_df = films_names_user405.merge(ratings_pivot_table, how='left', on='Film_id')\
                              .merge(genre_df, how='left', on='Film_id')
model_df = model_df[['Release_year', 'sum', 'count', 'Action', 'Adventure', 'Animation', 'Child', 'Comedy', 'Detective', 'Docs', 'Drama', 'Fantasy',\
                     'Noir', 'Horror', 'Musical', 'Mystiс', 'Romantic', 'Science', 'Thriller', 'War', 'Western', 'Rating']]
model_df

,Release_year,sum,count,Action,Adventure,Animation,Child,Comedy,Detective,Docs,Drama,Fantasy,Noir,Horror,Musical,Mystiс,Romantic,Science,Thriller,War,Western,Rating
0,1994,1600,394,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,4
1,1995,30,9,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1
2,1947,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
3,1991,252,65,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1
4,1995,108,32,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,1995,45,23,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
733,1945,77,22,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,4
734,1995,11,7,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
735,1989,983,251,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1


In [26]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 737 entries, 0 to 736
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Release_year  737 non-null    int64
 1   sum           737 non-null    int64
 2   count         737 non-null    int64
 3   Action        737 non-null    int64
 4   Adventure     737 non-null    int64
 5   Animation     737 non-null    int64
 6   Child         737 non-null    int64
 7   Comedy        737 non-null    int64
 8   Detective     737 non-null    int64
 9   Docs          737 non-null    int64
 10  Drama         737 non-null    int64
 11  Fantasy       737 non-null    int64
 12  Noir          737 non-null    int64
 13  Horror        737 non-null    int64
 14  Musical       737 non-null    int64
 15  Mystiс        737 non-null    int64
 16  Romantic      737 non-null    int64
 17  Science       737 non-null    int64
 18  Thriller      737 non-null    int64
 19  War           737 non-null   

## 5. Определение признаков

In [79]:
df = model_df[['Release_year', 'sum', 'count', 'Action', 'Adventure', 'Animation', 'Child', 'Comedy', 'Detective', 'Docs',\
               'Drama', 'Fantasy', 'Noir', 'Horror', 'Musical', 'Mystiс', 'Romantic', 'Science', 'Thriller', 'War', 'Western']]
res = model_df.Rating

In [80]:
from sklearn.model_selection import train_test_split

In [81]:
X_train, X_test, y_train, y_test = train_test_split(df, res, test_size=0.30) # разделяю выборку на тренировочную и тестовую
y_test = y_test.reset_index().Rating # сбрасываю индекс y_test

## 6.1 Построение модели Линейной Регрессии

In [82]:
from sklearn.linear_model import LinearRegression

In [83]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [84]:
lr.coef_  # коэфф при X

array([-1.10945233e-02, -2.78301794e-03,  2.02225558e-02, -8.56204699e-02,
       -2.76650629e-01, -1.31753951e-01,  3.28575625e-01,  2.12892805e-01,
        2.69993326e-01,  1.86344553e-01,  1.22475265e-01, -5.00911338e-02,
       -5.65889009e-01,  6.37280498e-02,  2.42453731e-01,  2.10790069e-01,
       -1.48685589e-01, -3.02565045e-01,  1.56195565e-01, -3.69541948e-01,
        5.17885818e-04])

In [85]:
lr.intercept_ # независимый коэф.

23.100331654809548

In [86]:
y_pred = lr.predict(X_test)  # массив прогнозов
y_pred

array([2.13269766, 1.2633266 , 1.98600491, 1.15060451, 1.38402036,
       1.71677358, 1.21284594, 2.03931908, 1.48842791, 2.23614294,
       1.58583832, 1.65506931, 1.23579816, 1.34158398, 1.20339751,
       3.26109587, 1.76741595, 2.34863206, 1.44045443, 1.20495546,
       1.37916677, 3.82990519, 0.77553408, 2.3999301 , 1.12964053,
       1.85346008, 1.19273083, 1.4322558 , 2.82386365, 1.98529848,
       1.19813607, 1.1339815 , 1.11608347, 1.44691721, 1.65893019,
       1.6367717 , 2.00552358, 1.26787177, 2.57843553, 2.56050525,
       1.45896513, 2.84013839, 1.74695878, 1.53460916, 2.42097203,
       0.61465529, 2.61326571, 1.88564127, 1.21906251, 1.45014119,
       1.56120083, 2.06644087, 1.61062102, 2.47068367, 1.4923596 ,
       0.91954075, 2.02932694, 1.78386003, 1.35122898, 1.23196912,
       0.93645487, 2.52746558, 1.73533092, 1.75402718, 1.57559542,
       1.55498045, 1.44813224, 3.16973196, 1.70451648, 3.77506773,
       1.16407352, 3.49595497, 0.92510679, 1.54144965, 1.66167

## 7.1 Оценка качества модели Линейной Регрессии

In [87]:
lr.score(X_test, y_test) # значение коэффициента детерминации

0.33855530325029115

In [88]:
# разница факта и прогноза
result_lr = pd.DataFrame([y_test, lr.predict(X_test)]).T
result_lr.columns = ['Rating_test', 'Rating_pred']
result_lr.Rating_pred = round(result_lr.Rating_pred)
result_lr.Rating_test = result_lr.Rating_test.astype('Int64')
result_lr.Rating_pred = result_lr.Rating_pred.astype('Int64')
result_lr

,Rating_test,Rating_pred
0,1,2
1,1,1
2,1,2
3,1,1
4,1,1
...,...,...
217,1,1
218,4,3
219,1,2
220,1,2


In [89]:
from sklearn.metrics import mean_squared_error

In [90]:
mean_squared_error(y_test, lr.predict(X_test))

1.0528706074043888

In [91]:
mean_squared_error(y_train, lr.predict(X_train))

1.2580186629127892

## 6.2 Построение модели Дерева

In [72]:
from sklearn.tree import DecisionTreeRegressor

In [73]:
tree=DecisionTreeRegressor()
tree.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

## 7.2 Оценка качества модели Дерева

In [74]:
tree.score(X_test, y_test) # значение коэффициента детерминации

-0.47528668473026525

In [76]:
# разница факта и прогноза
result_tree = pd.DataFrame([y_test, tree.predict(X_test)]).T
result_tree.columns = ['Rating_test', 'Rating_pred']
result_tree

,Rating_test,Rating_pred
0,1.0,1.0
1,1.0,1.0
2,5.0,4.0
3,1.0,1.0
4,1.0,1.0
...,...,...
217,1.0,3.0
218,1.0,1.0
219,2.0,1.0
220,5.0,1.0


In [77]:
mean_squared_error(y_test, tree.predict(X_test))

2.331081081081081

In [78]:
mean_squared_error(y_train, tree.predict(X_train))

0.008737864077669903